# xresnet

> Xresnet models. As in fastai v1, refactored.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
import torch
from model_constructor.base_constructor import Body

In [ ]:
import torch.nn as nn
from collections import OrderedDict

In [ ]:
from model_constructor.base_constructor import Net
from model_constructor.layers import ConvLayer, Noop, act_fn

# Xresnet constructor

In [ ]:
class DownsampleLayer(nn.Sequential):
    """Downsample layer for Xresnet Resblock"""
    def __init__(self, conv_layer, ni, nf, stride, act,
                 pool=nn.AvgPool2d(2, ceil_mode=True), pool_1st=True,
                 **kwargs):
        layers = [] if stride == 1 else [('pool', pool)]
        layers += [] if ni == nf else [('idconv', conv_layer(ni, nf, 1, act=act, **kwargs))]
        if not pool_1st:
            layers.reverse()
        super().__init__(OrderedDict(layers))

In [ ]:
class XResBlock(nn.Module):
    def __init__(self, ni, nh, expansion=1, stride=1, zero_bn=True,
                 conv_layer=ConvLayer, act_fn=act_fn, **kwargs):
        super().__init__()
        nf, ni = nh * expansion, ni * expansion
        layers = [('conv_0', conv_layer(ni, nh, 3, stride=stride, act_fn=act_fn, **kwargs)),
                  ('conv_1', conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, act_fn=act_fn, **kwargs))
                  ] if expansion == 1 else [
                      ('conv_0', conv_layer(ni, nh, 1, act_fn=act_fn, **kwargs)),
                      ('conv_1', conv_layer(nh, nh, 3, stride=stride, act_fn=act_fn, **kwargs)),
                      ('conv_2', conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False, act_fn=act_fn, **kwargs))
        ]
        self.convs = nn.Sequential(OrderedDict(layers))
        self.identity = DownsampleLayer(conv_layer, ni, nf, stride,
                                        act=False, act_fn=act_fn, **kwargs) if ni != nf or stride == 2 else Noop()
        self.merge = Noop()
        self.act_fn = act_fn

    def forward(self, x):
        return self.act_fn(self.merge(self.convs(x) + self.identity(x)))

In [ ]:
body = Body(XResBlock, expansion=4)
body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 2 blocks, expansion 4.
    (block_0): XResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity): DownsampleLayer(
        (idconv): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=

In [ ]:
xb = torch.randn(16, 64, 32, 32)
y = body(xb)
y.shape

torch.Size([16, 2048, 4, 4])

# Xresnet models.

In [ ]:
def xresnet18(**kwargs):
    """Constructs a xresnet-18 model. """
    return Net(stem_sizes=[32, 32], block=XResBlock, blocks=[2, 2, 2, 2], expansion=1, **kwargs)

def xresnet34(**kwargs):
    """Constructs axresnet-34 model. """
    return Net(stem_sizes=[32, 32], block=XResBlock, blocks=[3, 4, 6, 3], expansion=1, **kwargs)

def xresnet50(**kwargs):
    """Constructs axresnet-34 model. """
    return Net(stem_sizes=[32, 32], block=XResBlock, blocks=[3, 4, 6, 3], expansion=4, **kwargs)

In [ ]:
xresnet18()

Net(
  (stem): Stem(
    sizes: [3, 32, 32, 64]
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 2 blocks, expansion 1.
      (block_0): XResBloc

In [ ]:
xresnet34()

Net(
  (stem): Stem(
    sizes: [3, 32, 32, 64]
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 3 blocks, expansion 1.
      (block_0): XResBloc

In [ ]:
model = xresnet50()

In [ ]:
model.stem

Stem(
  sizes: [3, 32, 32, 64]
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
model.body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 3 blocks, expansion 4.
    (block_0): XResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity): DownsampleLayer(
        (idconv): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=

In [ ]:
model.head

Head(
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flat): Flatten()
  (fc): Linear(in_features=2048, out_features=1000, bias=True)
)

In [ ]:
xb = torch.randn(8, 3, 128, 128)
y = model(xb)
y.shape

torch.Size([8, 1000])

# end. xresnet.
model_constructor
by ayasyrev